#### Modelado de colas de espera en un hospital público de Guatemala
Integrantes: Stefano Aragoni, Luis Diego Santos, Carol Arevalo, Diego Perdomo



El sistema de salud pública en Guatemala, como en muchos otros países, enfrenta desafíos significativos. Uno de los más críticos es la prolongada espera que los pacientes deben soportar antes de recibir atención médica. Estas colas de espera pueden tener consecuencias graves, incluida una atención médica de menor calidad y una experiencia del paciente desfavorable. Este proyecto busca abordar este problema a través de técnicas avanzadas de modelado y simulación.

Hospital Roosvelt de Guatemala

- Cuenta con tres emergencias maternidad, adultos y pediatria
- Tiene capacidad para 950 pacientes en cama
- Se tine una capcidad de 50 pacientes en el area de emergencias
- Hay nueve camas para observación, 20 bancas para inhaloterapia y cuatro cuartos de shock.
- Diariamente atienden a un promedio de 2120 pacientes. 
- El 68.4% de los pacientes son consulta externa
- El 23.6% de los pacientes son de urgencias
- El 3.3% de los pacientes son hospitalizados
- El 2.4% de los pacientes son partos
- El 1.6% de los pacientes son cirugias selectivas
- El 0.4% de los pacientes entran a urgencias por traumas. 
- El 0.3% de los pacientes entran por obstetricia. 


Al entrar a urgencias:
1. Médicos evalúan si son casos que requieren atención de urgencia
2. Hay 4 clinicas de clasificacion. 
3. El 60% de los casos que entran a emergencias son *emfermedades comunes*. Estos son tratados en la clínica, con un tiempo de espera de 15 minutos.
4. Las *urgencias* son heridas, elevación de presión, deshidratación, o casos que merecen una atención mayor, estos pasan con un especialista, dijo el facultativo.
5. Cuando la persona llega con una herida de arma blanca o de fuego, con traumatismo, politraumatismo o complicaciones de enfermedades crónicas, se trata de una emergencia e ingresan cuanto antes para ser tratada en el área de Traumatología, Medicina interna o Cirugía.Estas personas entran a clinica de *claisificacion 1* directamente (desde ambulancias)
4. El tiempo de espera promedio era de 4 horas





#### Simulación del flujo de pacientes en urgencias en el Hospital Roosevelt de Guatemala.

##### 1. Definiciones y Parámetros Iniciales:

Empezamos por definir las constantes y parámetros basados en la información que has proporcionado:

In [25]:
import random

# Capacidades y parámetros del hospital
CAPACIDAD_EMERGENCIAS = 50
PACIENTES_POR_DIA = 2120
PORCENTAJE_URGENCIAS = 0.236

# Probabilidades para clasificación
PROB_ENFERMEDADES_COMUNES = 0.60
PROB_URGENCIAS_ESPECIALISTA = 1 - PROB_ENFERMEDADES_COMUNES

# Tiempo de espera
TIEMPO_ESPERA_COMUNES = 15 / 60  # Convertimos minutos a horas
TIEMPO_ESPERA_PROMEDIO = 4  # en horas


#### 2. Funciones de Simulación:

Las siguientes funciones nos ayudarán a simular la llegada de pacientes y su procesamiento en emergencias:

In [26]:
def clasificar_paciente():
    """
    Clasifica al paciente en base a probabilidades y regresa el tiempo de espera correspondiente.
    """
    rand_num = random.random()
    
    if rand_num <= PROB_ENFERMEDADES_COMUNES:
        return TIEMPO_ESPERA_COMUNES
    else:
        return TIEMPO_ESPERA_PROMEDIO

def simular_dia():
    """
    Simula un día en el hospital y regresa una lista con los tiempos de espera de los pacientes en urgencias.
    """
    num_pacientes_urgencias = int(PACIENTES_POR_DIA * PORCENTAJE_URGENCIAS)
    tiempos_espera = []

    for _ in range(num_pacientes_urgencias):
        tiempo_espera = clasificar_paciente()
        tiempos_espera.append(tiempo_espera)

    return tiempos_espera


3. Ejecución y Análisis:
Ahora, podemos ejecutar la simulación y obtener algunos resultados de interés:



In [27]:
def main():
    tiempos_espera = simular_dia()
    
    promedio_espera = sum(tiempos_espera) / len(tiempos_espera)
    pacientes_sobre_4_horas = sum(1 for t in tiempos_espera if t >= 4)

    print(f"Promedio de tiempo de espera: {promedio_espera:.2f} horas")
    print(f"Pacientes que esperaron más de 4 horas: {pacientes_sobre_4_horas}")

if __name__ == "__main__":
    main()


Promedio de tiempo de espera: 1.87 horas
Pacientes que esperaron más de 4 horas: 216


#### Simulación basada en agentes del flujo de pacientes en urgencias en el Hospital Roosevelt de Guatemala.

#### 1. Definición de Agentes:

Definiremos dos clases básicas: Paciente y Clinica.

In [31]:
from mesa import Agent
import random

class Paciente(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.tiempo_espera = 0
        self.estado = 'entrando'
        self.clinica_asignada = None

    def step(self):
        if self.estado == 'entrando':
            clinica_clasificadora_disponible = self.model.clinica_disponible(tipo_clinica=ClinicaClasificadora)
            if clinica_clasificadora_disponible:
                self.clinica_asignada = clinica_clasificadora_disponible
                self.clinica_asignada.atender_paciente()
                clasificacion = self.clinica_asignada.clasificar_paciente()

                if clasificacion == 'enfermedad_comun':
                    clinica_target = self.model.clinica_disponible(tipo_clinica=ClinicaEnfermedadesComunes)
                elif clasificacion == 'urgencia':
                    clinica_target = self.model.clinica_disponible(tipo_clinica=ClinicaUrgencias)
                elif clasificacion == 'clasificacion_1':
                    clinica_target = self.model.clinica_disponible(tipo_clinica=ClinicaClasificacion1)

                if clinica_target: # Si hay una clínica del tipo correcto disponible
                    self.clinica_asignada.liberar()  # Libera la clínica clasificadora
                    self.clinica_asignada = clinica_target
                    self.clinica_asignada.atender_paciente()
                    self.estado = clasificacion

                    if clasificacion == 'enfermedad_comun':
                        self.model.pacientes_enfermedad_comun += 1
                        self.model.tiempo_espera_enfermedad_comun += clinica_target.tiempo_espera()
                    elif clasificacion == 'urgencia':
                        self.model.pacientes_urgencias += 1
                        self.model.tiempo_espera_urgencias += clinica_target.tiempo_espera()
                    elif clasificacion == 'clasificacion_1':
                        self.model.pacientes_clasificacion_1 += 1
                        self.model.tiempo_espera_clasificacion_1 += clinica_target.tiempo_espera()


class Clinica(Agent):
    # Esta clase actúa como una clase base o "padre" para las clínicas específicas.
    def __init__(self, unique_id, model, capacidad=1):
        super().__init__(unique_id, model)
        self.capacidad = capacidad
        self.pacientes_actualmente = 0

    def atender_paciente(self):
        self.pacientes_actualmente += 1

    def liberar(self):
        self.pacientes_actualmente -= 1
        if self.pacientes_actualmente < 0:
            self.pacientes_actualmente = 0

    def esta_disponible(self):
        return self.pacientes_actualmente < self.capacidad

class ClinicaEnfermedadesComunes(Clinica):
    def __init__(self, unique_id, model, capacidad=10):  # capacidad default es 10
        super().__init__(unique_id, model, capacidad)
    
    def tiempo_espera(self):
        return random.randint(5, 30) 

class ClinicaUrgencias(Clinica):
    def __init__(self, unique_id, model, capacidad=5):  # capacidad default es 5
        super().__init__(unique_id, model, capacidad)
    
    def tiempo_espera(self):
        # Aquí puedes definir el tiempo de espera promedio o alguna lógica adicional
        return random.randint(30, 180) 

class ClinicaClasificacion1(Clinica):
    def __init__(self, unique_id, model, capacidad=3):  # capacidad default es 3
        super().__init__(unique_id, model, capacidad)
    
    def tiempo_espera(self):
        # Aquí puedes definir el tiempo de espera promedio o alguna lógica adicional
        return random.randint(10, 60)  # Por ejemplo, entre 10 minutos a 1 hora
    
class ClinicaClasificadora(Clinica):
    def clasificar_paciente(self):
        r = random.random()
        if r < 0.60:
            return 'enfermedad_comun'
        elif r < 0.90:  # 30% de probabilidad (60% + 30% = 90%)
            return 'urgencia'
        else:
            return 'clasificacion_1'

    

#### 2. Definición del Modelo:

El modelo controlará la lógica global de la simulación y gestionará todos los agentes.

In [32]:
from mesa import Model
from mesa.time import RandomActivation

class HospitalModel(Model):
    def __init__(self, N, num_clinicas_clasificadoras=4, num_clinicas_comunes=2, num_clinicas_urgencias=1, num_clinicas_clasificacion1=1):
        self.num_pacientes = N
        self.schedule = RandomActivation(self)

        # Iniciar contadores y acumuladores de tiempo de espera
        self.pacientes_enfermedad_comun = 0
        self.pacientes_urgencias = 0
        self.pacientes_clasificacion_1 = 0
        self.tiempo_espera_enfermedad_comun = 0
        self.tiempo_espera_urgencias = 0
        self.tiempo_espera_clasificacion_1 = 0


        # Crear clínicas
        self.clinicas_clasificadoras = [ClinicaClasificadora(i, self) for i in range(num_clinicas_clasificadoras)]
        self.clinicas_comunes = [ClinicaEnfermedadesComunes(i, self) for i in range(num_clinicas_comunes)]
        self.clinicas_urgencias = [ClinicaUrgencias(i+len(self.clinicas_comunes), self) for i in range(num_clinicas_urgencias)]
        self.clinicas_clasificacion1 = [ClinicaClasificacion1(i+len(self.clinicas_comunes)+len(self.clinicas_urgencias), self) for i in range(num_clinicas_clasificacion1)]
        self.clinicas = self.clinicas_comunes + self.clinicas_urgencias + self.clinicas_clasificacion1 + self.clinicas_clasificadoras

        # Crear pacientes
        for i in range(self.num_pacientes):
            paciente = Paciente(i, self)
            self.schedule.add(paciente)

    def reporte(self):
        print(f"Cantidad de pacientes que entraron: {self.num_pacientes}")
        print(f"Cantidad de pacientes de urgencias: {self.pacientes_urgencias}")
        print(f"Cantidad de pacientes de enfermedad común: {self.pacientes_enfermedad_comun}")
        print(f"Cantidad de pacientes de clasificación 1: {self.pacientes_clasificacion_1}")
        if self.pacientes_enfermedad_comun != 0:
            print(f"Promedio de tiempo de espera enfermedad común: {self.tiempo_espera_enfermedad_comun / self.pacientes_enfermedad_comun}")
        if self.pacientes_urgencias != 0:
            print(f"Promedio de tiempo de espera urgencias: {self.tiempo_espera_urgencias / self.pacientes_urgencias}")
        if self.pacientes_clasificacion_1 != 0:
            print(f"Promedio de tiempo de espera clasificación 1: {self.tiempo_espera_clasificacion_1 / self.pacientes_clasificacion_1}")

    def step(self):
        # Avanza un paso en la simulación
        self.schedule.step()

    def clinica_disponible(self, tipo_clinica=None):
        for clinica in self.clinicas:
            if isinstance(clinica, tipo_clinica) and clinica.esta_disponible():
                return clinica
        return None


### 3. Ejecución:

Se crear una instancia del modelo y se ejecuta:

In [33]:
model = HospitalModel(100)  # Por ejemplo, 100 pacientes

for i in range(10):  # Ejecuta la simulación durante 10 pasos
    model.step()

model.reporte()


Cantidad de pacientes que entraron: 100
Cantidad de pacientes de urgencias: 5
Cantidad de pacientes de enfermedad común: 14
Cantidad de pacientes de clasificación 1: 3
Promedio de tiempo de espera enfermedad común: 16.642857142857142
Promedio de tiempo de espera urgencias: 55.4
Promedio de tiempo de espera clasificación 1: 28.0
